# **Testing Bell´s Inequality with Qiskit**

In [ ]:
!pip install pylatexenc


Para ilustrar cómo se puede hacer el experimento de Bell en una computadora cuántica requerimos entender cómo hacer una medición en bases rotadas a ángulos arbitrarios $\theta$ y $\phi$.

Partiendo de los 2 estados cuánticos ortogonales 
$$
|\psi\rangle = \cos({\frac{\theta}{2}})|0\rangle +e^{i\phi} \sin({\frac{\theta}{2}})|1\rangle=\begin{pmatrix} \cos({\frac{\theta}{2}}) \\\\ e^{i\phi}\sin({\frac{\theta}{2}}) \end{pmatrix} 
$$
y

$$
|\psi^{'}\rangle =\cos({\frac{\pi-\theta}{2}})|0\rangle +e^{i(\pi+\phi)} \sin({\frac{\pi-\theta}{2}})|1\rangle=\begin{pmatrix} \sin({\frac{\theta}{2}}) \\\\ -e^{i\phi}\cos({\frac{\theta}{2}}) \end{pmatrix} 
$$

vemos que los operadores unitarios de rotación 


$$R_Y(\theta) = \exp\left(-i \frac{\theta}{2} Y\right) =
    \begin{pmatrix}
        \cos{\frac{\theta}{2}} & -\sin{\frac{\theta}{2}} \\
        \sin{\frac{\theta}{2}} & \cos{\frac{\theta}{2}}
    \end{pmatrix}
$$

y

$$R_Z(\phi) = \exp\left(-i \frac{\phi}{2} Z\right) =
    \begin{pmatrix}
        e^{-i\phi/2} & 0 \\
       0 & e^{i\phi/2}
    \end{pmatrix}
$$



transforman la base  $|\psi\rangle, |\psi^{'}\rangle$ en la base computacional
$|0\rangle, |1\rangle$ (salvo una fase)
$$
R_Y(-\theta) R_Z(-\phi)|\psi\rangle =\begin{pmatrix}
        \cos{\frac{\theta}{2}} & \sin{\frac{\theta}{2}} \\
        -\sin{\frac{\theta}{2}} & \cos{\frac{\theta}{2}}
    \end{pmatrix}
    \begin{pmatrix}
        e^{i\phi/2} & 0 \\
       0 & e^{-i\phi/2}
    \end{pmatrix}
    \begin{pmatrix} \cos({\frac{\theta}{2}}) \\\\ e^{i\phi}\sin({\frac{\theta}{2}}) \end{pmatrix} =e^{i\phi/2}
    \begin{pmatrix} 1 \\\\ 0 \end{pmatrix}
    =e^{i\phi/2}|0\rangle
$$
$$
R_Y(-\theta) R_Z(-\phi)|\psi^{'}\rangle =\begin{pmatrix}
        \cos{\frac{\theta}{2}} & \sin{\frac{\theta}{2}} \\
        -\sin{\frac{\theta}{2}} & \cos{\frac{\theta}{2}}
    \end{pmatrix}
    \begin{pmatrix}
        e^{i\phi/2} & 0 \\
       0 & e^{-i\phi/2}
    \end{pmatrix}
    \begin{pmatrix} \sin({\frac{\theta}{2}}) \\\\ -e^{i\phi}\cos({\frac{\theta}{2}}) \end{pmatrix}  =e^{i\phi/2}
    \begin{pmatrix} 0 \\\\ 1 \end{pmatrix}
    =e^{i\phi/2}|1\rangle
$$

De modo que para medir un qubit en la base $|\psi\rangle, |\psi^{'}\rangle$ basta con aplicar $R_Y(-\theta) R_Z(-\phi)$ a ese qubit justo antes de medirlo en la base computacional $|0\rangle, |1\rangle$ de Qiskit.

Con estas herramientas ya podemos programar el experimento de la desigualdad de Bell que vimos en la sección anterior.

Primero generamos el estado singlete de Bell

In [ ]:
from qiskit import QuantumCircuit, Aer,execute
import qiskit.quantum_info as qi
import numpy as np
qc = QuantumCircuit(2,2) # Ponemos 2 qubits y 2 bits
qc.h(0) 
qc.cx(0,1) 
qc.x(1)
qc.z(1)
qc.barrier()
#qc.ry(-2*np.pi/3,1)
stv = qi.Statevector.from_instruction(qc)
#qc.rz(0,1)
#qc.ry(-2*np.pi/3,1)
#qc.measure(0,0)
#qc.measure(1,1)
qc.draw("mpl")


In [ ]:
stv.draw('latex', prefix="|\\psi\\rangle =")

In [ ]:
backend = Aer.get_backend('qasm_simulator')
N_corr = 0
N_anticorr = 0
qc3 = QuantumCircuit(2, 2)
qc3.measure(0,0)
qc3.measure(1,1)
print("                       Experimento A1B1")
qc11=qc
print(qc11)
qc11=qc.compose(qc3)
job11 = execute(qc11,backend, shots=1000)
result = job11.result()
print("Resultados :",result.get_counts(qc11))
if job11.result().get_counts().get('11') is not None:
  N_corr += job11.result().get_counts()['11']
if job11.result().get_counts().get('00') is not None:
  N_corr += job11.result().get_counts()['00']
N_anticorr += job11.result().get_counts()['01']+job11.result().get_counts()['10']

print()
print("                       Experimento A1B2")
qc12 = QuantumCircuit(2, 2)
qc12.rz(0,1)
qc12.ry(-2*np.pi/3,1)
qc12.barrier()
qc12=qc.compose(qc12)
qc12=qc12.compose(qc3)
print(qc12)
job12 = execute(qc12,backend, shots=1000)
result = job12.result()
print("Resultados :",result.get_counts(qc12))
N_corr += job12.result().get_counts()['11']+job12.result().get_counts()['00']
N_anticorr += job12.result().get_counts()['01']+job12.result().get_counts()['10']

print()
print("                       Experimento A1B3")
qc13 = QuantumCircuit(2, 2)
qc13.rz(-np.pi,1)
qc13.ry(-2*np.pi/3,1)
qc13.barrier()
qc13=qc.compose(qc13)
qc13=qc13.compose(qc3)
print(qc13)
job13 = execute(qc13,backend, shots=1000)
result = job13.result()
print("Resultados :",result.get_counts(qc13))
N_corr += job13.result().get_counts()['11']+job13.result().get_counts()['00']
N_anticorr += job13.result().get_counts()['01']+job13.result().get_counts()['10']

print()
print("                       Experimento A2B1")
qc21 = QuantumCircuit(2, 2)
qc21.rz(0,0)
qc21.ry(-2*np.pi/3,0)
qc21.barrier()
qc21=qc.compose(qc21)
qc21=qc21.compose(qc3)
print(qc21)
job21 = execute(qc21,backend, shots=1000)
result = job21.result()
print("Resultados :",result.get_counts(qc21))
N_corr += job21.result().get_counts()['11']+job21.result().get_counts()['00']
N_anticorr += job21.result().get_counts()['01']+job21.result().get_counts()['10']

print()
print("                       Experimento A2B2")
qc22 = QuantumCircuit(2, 2)
qc22.rz(0,0)
qc22.ry(-2*np.pi/3,0)
qc22.rz(0,1)
qc22.ry(-2*np.pi/3,1)
qc22.barrier()
qc22=qc.compose(qc22)
qc22=qc22.compose(qc3)
print(qc22)
job22 = execute(qc22,backend, shots=1000)
result = job22.result()
print("Resultados :",result.get_counts(qc22))
if job22.result().get_counts().get('11') is not None:
  N_corr += job22.result().get_counts()['11']
if job22.result().get_counts().get('00') is not None:
  N_corr += job22.result().get_counts()['00']
N_anticorr += job22.result().get_counts()['01']+job22.result().get_counts()['10']

print()
print("                       Experimento A2B3")
qc23 = QuantumCircuit(2, 2)
qc23.rz(0,0)
qc23.ry(-2*np.pi/3,0)
qc23.rz(-np.pi,1)
qc23.ry(-2*np.pi/3,1)
qc23.barrier()
qc23=qc.compose(qc23)
qc23=qc23.compose(qc3)
print(qc23)
job23 = execute(qc23,backend, shots=1000)
result = job23.result()
print("Resultados :",result.get_counts(qc23))
N_corr += job23.result().get_counts()['11']+job23.result().get_counts()['00']
N_anticorr += job23.result().get_counts()['01']+job23.result().get_counts()['10']

print()
print("                       Experimento A3B1")
qc31 = QuantumCircuit(2, 2)
qc31.rz(-np.pi,0)
qc31.ry(-2*np.pi/3,0)
qc31.barrier()
qc31=qc.compose(qc31)
qc31=qc31.compose(qc3)
print(qc31)
job31 = execute(qc31,backend, shots=1000)
result = job31.result()
print("Resultados :",result.get_counts(qc31))
N_corr += job31.result().get_counts()['11']+job31.result().get_counts()['00']
N_anticorr += job31.result().get_counts()['01']+job31.result().get_counts()['10']

print()
print("                       Experimento A3B2")
qc32 = QuantumCircuit(2, 2)
qc32.rz(-np.pi,0)
qc32.ry(-2*np.pi/3,0)
qc32.rz(0,1)
qc32.ry(-2*np.pi/3,1)
qc32.barrier()
qc32=qc.compose(qc32)
qc32=qc32.compose(qc3)
print(qc32)
job32 = execute(qc32,backend, shots=1000)
result = job31.result()
print("Resultados :",result.get_counts(qc32))
N_corr += job32.result().get_counts()['11']+job32.result().get_counts()['00']
N_anticorr += job32.result().get_counts()['01']+job32.result().get_counts()['10']

print()
print("                       Experimento A3B3")
qc33 = QuantumCircuit(2, 2)
qc33.rz(-np.pi,0)
qc33.ry(-2*np.pi/3,0)
qc33.rz(-np.pi,1)
qc33.ry(-2*np.pi/3,1)
qc33.barrier()
qc33=qc.compose(qc33)
qc33=qc33.compose(qc3)
print(qc33)
job33 = execute(qc33,backend, shots=1000)
result = job33.result()
print("Resultados :",result.get_counts(qc33))
if job33.result().get_counts().get('11') is not None:
  N_corr += job33.result().get_counts()['11']
if job33.result().get_counts().get('00') is not None:
  N_corr += job33.result().get_counts()['00']
N_anticorr += job33.result().get_counts()['01']+job33.result().get_counts()['10']
print("Total correlated:", N_corr, " Total anti-correlated:", N_anticorr)